In [87]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
import kagglehub
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
!pip install imblearn --break-system-packages
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from collections import Counter

Defaulting to user installation because normal site-packages is not writeable


In [63]:
path = kagglehub.dataset_download("balaka18/email-spam-classification-dataset-csv")

In [64]:
import os
file= [f for f in os.listdir(path) if f.endswith('.csv')][0]

In [65]:
df = pd.read_csv(file)
df.head()

,Email No.,the,to,ect,and,for,of,a,you,hou,...,connevey,jay,valued,lay,infrastructure,military,allowing,ff,dry,Prediction
0,Email 1,0,0,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Email 2,8,13,24,6,6,2,102,1,27,...,0,0,0,0,0,0,0,1,0,0
2,Email 3,0,0,1,0,0,0,8,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Email 4,0,5,22,0,5,1,51,2,10,...,0,0,0,0,0,0,0,0,0,0
4,Email 5,7,6,17,1,5,2,57,0,9,...,0,0,0,0,0,0,0,1,0,0


In [66]:
print("\nMissing values:")
print(df.isnull().sum())
print("\nData types:")
print(df.dtypes)


Missing values:
Email No.     0
the           0
to            0
ect           0
and           0
             ..
military      0
allowing      0
ff            0
dry           0
Prediction    0
Length: 3002, dtype: int64

Data types:
Email No.     object
the            int64
to             int64
ect            int64
and            int64
               ...  
military       int64
allowing       int64
ff             int64
dry            int64
Prediction     int64
Length: 3002, dtype: object


In [67]:
#print("\nUnique values in each column:")
#for col in df.columns:
#    print(f"{col}: {df[col].nunique()} unique values")
#    if df[col].nunique() < 10:  # Show unique values for categorical columns
#        print(f"  Values: {df[col].unique()}")

In [68]:
print(f"\nClass distribution in 'Prediction':")
print(df['Prediction'].value_counts())
print(f"\nClass distribution (percentage):")
class_dist = df['Prediction'].value_counts(normalize=True) * 100
print(class_dist)


Class distribution in 'Prediction':
Prediction
0    3672
1    1500
Name: count, dtype: int64

Class distribution (percentage):
Prediction
0    70.99768
1    29.00232
Name: proportion, dtype: float64


In [69]:
#Data Preparation
feature_cols = [col for col in df.columns if col not in ['Email No.', 'Prediction']]
X = df[feature_cols]  # Features (word frequencies)
y = df['Prediction']  # Target (0=Ham, 1=Spam)print(f"✓ Features shape: {X.shape}")
print(f"✓ Target shape: {y.shape}")
print(f"✓ Number of features: {len(feature_cols)}")

# Check data types and any potential issues
print(f"✓ Feature data type: {X.dtypes[0]}")
print(f"✓ Target data type: {y.dtype}")
print(f"✓ Any missing values in features: {X.isnull().sum().sum()}")
print(f"✓ Any missing values in target: {y.isnull().sum()}")

✓ Target shape: (5172,)
✓ Number of features: 3000
✓ Feature data type: int64
✓ Target data type: int64
✓ Any missing values in features: 0
✓ Any missing values in target: 0


/tmp/ipykernel_302219/1769405930.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(f"✓ Feature data type: {X.dtypes[0]}")


In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2,random_state=42,stratify=y)
print(f"Training set size: {X_train.shape[0]} samples")
print(f"Test set size: {X_test.shape[0]} samples")
print(f"Training set class distribution:")
print(f"Ham (0): {(y_train == 0).sum()} ({(y_train == 0).mean():.1%})")
print(f"Spam (1): {(y_train == 1).sum()} ({(y_train == 1).mean():.1%})")

Training set size: 4137 samples
Test set size: 1035 samples
Training set class distribution:
Ham (0): 2937 (71.0%)
Spam (1): 1200 (29.0%)


In [71]:
from sklearn.preprocessing import StandardScaler
smote = SMOTE(random_state=42, k_neighbors=3)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [72]:
svm_model = SVC(kernel='rbf', random_state=42, probability=True)
lr_model = LogisticRegression(random_state=42, max_iter=10000)

In [73]:
import time
start_time = time.time()
svm_model.fit(X_train_scaled, y_train)
svm_train_time = time.time() - start_time
print(f"SVM training completed in {svm_train_time:.2f} seconds")

SVM training completed in 179.57 seconds


In [74]:
start_time = time.time()
lr_model.fit(X_train_scaled, y_train)
lr_train_time = time.time() - start_time
print(f"Logistic Regression training completed in {lr_train_time:.2f} seconds")

Logistic Regression training completed in 2.40 seconds


In [75]:
svm_train_pred = svm_model.predict(X_train_scaled)
svm_test_pred = svm_model.predict(X_test_scaled)
svm_test_proba = svm_model.predict_proba(X_test_scaled)[:, 1]

lr_train_pred = lr_model.predict(X_train_scaled)
lr_test_pred = lr_model.predict(X_test_scaled)
lr_test_proba = lr_model.predict_proba(X_test_scaled)[:, 1]

In [76]:
def evaluate_model(y_true, y_pred, model_name, dataset_type):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    print(f"\n{model_name} - {dataset_type} Performance:")
    print(f"  Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"  Precision: {precision:.4f} ({precision*100:.2f}%)")
    print(f"  Recall:    {recall:.4f} ({recall*100:.2f}%)")
    print(f"  F1-Score:  {f1:.4f} ({f1*100:.2f}%)")
    return accuracy, precision, recall, f1

In [77]:
print("\nSVM MODEL EVALUATION")
svm_train_acc, svm_train_prec, svm_train_rec, svm_train_f1 = evaluate_model(
    y_train, svm_train_pred, "SVM", "Training"
)
svm_test_acc, svm_test_prec, svm_test_rec, svm_test_f1 = evaluate_model(
    y_test, svm_test_pred, "SVM", "Test"
)

print("\nLOGISTIC REGRESSION MODEL EVALUATION")
lr_train_acc, lr_train_prec, lr_train_rec, lr_train_f1 = evaluate_model(
    y_train, lr_train_pred, "Logistic Regression", "Training"
)
lr_test_acc, lr_test_prec, lr_test_rec, lr_test_f1 = evaluate_model(
    y_test, lr_test_pred, "Logistic Regression", "Test"
)


SVM MODEL EVALUATION

SVM - Training Performance:
  Accuracy:  0.9683 (96.83%)
  Precision: 0.9981 (99.81%)
  Recall:    0.8925 (89.25%)
  F1-Score:  0.9424 (94.24%)

SVM - Test Performance:
  Accuracy:  0.9304 (93.04%)
  Precision: 0.9831 (98.31%)
  Recall:    0.7733 (77.33%)
  F1-Score:  0.8657 (86.57%)

LOGISTIC REGRESSION MODEL EVALUATION

Logistic Regression - Training Performance:
  Accuracy:  0.9998 (99.98%)
  Precision: 0.9992 (99.92%)
  Recall:    1.0000 (100.00%)
  F1-Score:  0.9996 (99.96%)

Logistic Regression - Test Performance:
  Accuracy:  0.9681 (96.81%)
  Precision: 0.9211 (92.11%)
  Recall:    0.9733 (97.33%)
  F1-Score:  0.9465 (94.65%)


In [78]:
#Now aggregating
classifiers = {
    'SVM': SVC(kernel='rbf', random_state=42, probability=True),
    'Logistic_Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random_Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'Naive_Bayes': MultinomialNB(),
    'AdaBoost': AdaBoostClassifier(random_state=42, n_estimators=100),
    'K_Neighbors': KNeighborsClassifier(n_neighbors=5),
    'Decision_Tree': DecisionTreeClassifier(random_state=42, max_depth=10)
}

In [79]:
trained_models = {}
training_times = {}
individual_results = {}

for name, clf in classifiers.items():
    print(f"\nTraining {name.replace('_', ' ')}...")
    start_time = time.time()
    
    if name == 'Naive_Bayes':
        clf.fit(X_train, y_train)
        train_pred = clf.predict(X_train)
        test_pred = clf.predict(X_test)
        test_proba = clf.predict_proba(X_test)[:, 1]
    else:
        clf.fit(X_train_scaled, y_train)
        train_pred = clf.predict(X_train_scaled)
        test_pred = clf.predict(X_test_scaled)
        test_proba = clf.predict_proba(X_test_scaled)[:, 1]
    
    training_time = time.time() - start_time
    trained_models[name] = clf
    training_times[name] = training_time
    
    train_acc = accuracy_score(y_train, train_pred)
    test_acc = accuracy_score(y_test, test_pred)
    test_prec = precision_score(y_test, test_pred)
    test_rec = recall_score(y_test, test_pred)
    test_f1 = f1_score(y_test, test_pred)
    
    individual_results[name] = {
        'train_accuracy': train_acc,
        'test_accuracy': test_acc,
        'test_precision': test_prec,
        'test_recall': test_rec,
        'test_f1': test_f1,
        'training_time': training_time,
        'predictions': test_pred,
        'probabilities': test_proba
    }
    
    print(f"Training completed in {training_time:.2f}s")
    print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")


Training SVM...
Training completed in 251.95s
Test Accuracy: 0.9304 (93.04%)

Training Logistic Regression...
Training completed in 2.36s
Test Accuracy: 0.9681 (96.81%)

Training Random Forest...
Training completed in 2.69s
Test Accuracy: 0.9652 (96.52%)

Training Naive Bayes...
Training completed in 0.20s
Test Accuracy: 0.9420 (94.20%)

Training AdaBoost...
Training completed in 17.82s
Test Accuracy: 0.9469 (94.69%)

Training K Neighbors...
Training completed in 31.92s
Test Accuracy: 0.8338 (83.38%)

Training Decision Tree...
Training completed in 0.96s
Test Accuracy: 0.8966 (89.66%)


In [80]:
performance_data = []
for name, results in individual_results.items():
    performance_data.append({
        'Model': name.replace('_', ' '),
        'Test Accuracy': f"{results['test_accuracy']:.4f}",
        'Test Precision': f"{results['test_precision']:.4f}",
        'Test Recall': f"{results['test_recall']:.4f}",
        'Test F1-Score': f"{results['test_f1']:.4f}",
        'Training Time': f"{results['training_time']:.2f}s"
    })

In [81]:
performance_df = pd.DataFrame(performance_data)
performance_df = performance_df.sort_values('Test Accuracy', ascending=False)
print("\nIndividual Model Performance (sorted by accuracy):")
print(performance_df.to_string(index=False))


Individual Model Performance (sorted by accuracy):
              Model Test Accuracy Test Precision Test Recall Test F1-Score Training Time
Logistic Regression        0.9681         0.9211      0.9733        0.9465         2.36s
      Random Forest        0.9652         0.9371      0.9433        0.9402         2.69s
           AdaBoost        0.9469         0.9125      0.9033        0.9079        17.82s
        Naive Bayes        0.9420         0.8681      0.9433        0.9042         0.20s
                SVM        0.9304         0.9831      0.7733        0.8657       251.95s
      Decision Tree        0.8966         0.7734      0.9100        0.8361         0.96s
        K Neighbors        0.8338         0.6435      0.9567        0.7694        31.92s


In [89]:
all_predictions = {}
for name, results in individual_results.items():
    all_predictions[name] = results['predictions']
prediction_matrix = np.column_stack([all_predictions[name] for name in classifiers.keys()])

def majority_vote(predictions_matrix):
    majority_predictions = []
    vote_counts = []
    
    for i in range(predictions_matrix.shape[0]):
        sample_predictions = predictions_matrix[i, :]
        vote_count = Counter(sample_predictions)
        majority_class = vote_count.most_common(1)[0][0]
        majority_predictions.append(majority_class)
        vote_counts.append(dict(vote_count))
    
    return np.array(majority_predictions), vote_counts

# Apply majority voting
ensemble_predictions, vote_details = majority_vote(prediction_matrix)

# Evaluate ensemble performance
ensemble_accuracy = accuracy_score(y_test, ensemble_predictions)
ensemble_precision = precision_score(y_test, ensemble_predictions)
ensemble_recall = recall_score(y_test, ensemble_predictions)
ensemble_f1 = f1_score(y_test, ensemble_predictions)

print("\n" + "="*50)
print("ENSEMBLE MODEL PERFORMANCE (Majority Voting)")
print("="*50)
print(f"Accuracy:  {ensemble_accuracy:.4f} ({ensemble_accuracy*100:.2f}%)")
print(f"Precision: {ensemble_precision:.4f} ({ensemble_precision*100:.2f}%)")
print(f"Recall:    {ensemble_recall:.4f} ({ensemble_recall*100:.2f}%)")
print(f"F1-Score:  {ensemble_f1:.4f} ({ensemble_f1*100:.2f}%)")

ensemble_data = {
    'Model': 'Ensemble (Majority Vote)',
    'Test Accuracy': f"{ensemble_accuracy:.4f}",
    'Test Precision': f"{ensemble_precision:.4f}",
    'Test Recall': f"{ensemble_recall:.4f}",
    'Test F1-Score': f"{ensemble_f1:.4f}",
    'Training Time': 'N/A'
}


ENSEMBLE MODEL PERFORMANCE (Majority Voting)
Accuracy:  0.9758 (97.58%)
Precision: 0.9421 (94.21%)
Recall:    0.9767 (97.67%)
F1-Score:  0.9591 (95.91%)
